# Amazon Bedrock 的知识库

## 概述

这将帮助你快速开始使用 [AmazonKnowledgeBaseRetriever](/docs/concepts/retrievers)。如需了解 AmazonKnowledgeBaseRetriever 所有功能和配置的详细文档，请前往 [API 参考](https://api.js.langchain.com/classes/langchain_aws.AmazonKnowledgeBaseRetriever.html)。

Amazon Bedrock 的知识库是由 Amazon Web Services (AWS) 提供的端到端 RAG 工作流程的完全托管支持。
它提供了一整套数据摄入流程，将你的文档转换为嵌入（向量），并将这些嵌入存储在专用的向量数据库中。
Amazon Bedrock 的知识库支持多种流行的向量存储数据库，包括 Amazon OpenSearch Serverless 的向量引擎、Pinecone、Redis Enterprise Cloud、Amazon Aurora（即将推出）和 MongoDB（即将推出）。

### 集成详情

| 检索器 | 自托管 | 云服务 | 包 | [Python 支持](https://python.langchain.com/docs/integrations/retrievers/bedrock/) |
| :--- | :--- | :---: | :---: | :---: |
[AmazonKnowledgeBaseRetriever](https://api.js.langchain.com/classes/langchain_aws.AmazonKnowledgeBaseRetriever.html) | 🟠 (详见下方) | ✅ | @langchain/aws | ✅ |

> AWS 知识库检索器可在你自己的 AWS 基础设施上运行，从这个意义上说它可以“自托管”。但无法在其他云提供商或本地环境中运行。

## 配置

要使用 AmazonKnowledgeBaseRetriever，你需要拥有一个 AWS 账户，以便管理你的索引和文档。在设置好账户后，设置以下环境变量：

```bash
process.env.AWS_KNOWLEDGE_BASE_ID=your-knowledge-base-id
process.env.AWS_ACCESS_KEY_ID=your-access-key-id
process.env.AWS_SECRET_ACCESS_KEY=your-secret-access-key
```

如果你想从单个查询中获得自动追踪，你也可以通过取消注释以下内容来设置你的 [LangSmith](https://docs.smith.langchain.com/) API 密钥：

In [ ]:
// process.env.LANGSMITH_API_KEY = "<YOUR API KEY HERE>";
// process.env.LANGSMITH_TRACING = "true";

### 安装

此检索器位于 `@langchain/aws` 包中：

```{=mdx}
import IntegrationInstallTooltip from "@mdx_components/integration_install_tooltip.mdx";
import Npm2Yarn from "@theme/Npm2Yarn";

<IntegrationInstallTooltip></IntegrationInstallTooltip>

<Npm2Yarn>
  @langchain/aws @langchain/core
</Npm2Yarn>
```

## 实例化

现在我们可以实例化检索器：

In [ ]:
import { AmazonKnowledgeBaseRetriever } from "@langchain/aws";

const retriever = new AmazonKnowledgeBaseRetriever({
  topK: 10,
  knowledgeBaseId: process.env.AWS_KNOWLEDGE_BASE_ID,
  region: "us-east-2",
  clientOptions: {
    credentials: {
      accessKeyId: process.env.AWS_ACCESS_KEY_ID,
      secretAccessKey: process.env.AWS_SECRET_ACCESS_KEY,
    },
  },
});

## 用法

In [ ]:
const query = "..."

await retriever.invoke(query);

## 在链式调用中使用

与其他检索器类似，AmazonKnowledgeBaseRetriever 可以通过[链式调用](/docs/how_to/sequence/)集成到 LLM 应用程序中。

我们需要一个 LLM 或聊天模型：

```{=mdx}
import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs customVarName="llm" />
```

In [ ]:
// @lc-docs-hide-cell

import { ChatOpenAI } from "@langchain/openai";

const llm = new ChatOpenAI({
  model: "gpt-4o-mini",
  temperature: 0,
});

In [ ]:
import { ChatPromptTemplate } from "@langchain/core/prompts";
import { RunnablePassthrough, RunnableSequence } from "@langchain/core/runnables";
import { StringOutputParser } from "@langchain/core/output_parsers";

import type { Document } from "@langchain/core/documents";

const prompt = ChatPromptTemplate.fromTemplate(`
Answer the question based only on the context provided.

Context: {context}

Question: {question}`);

const formatDocs = (docs: Document[]) => {
  return docs.map((doc) => doc.pageContent).join("\n\n");
}

// See https://js.langchain.com/docs/tutorials/rag
const ragChain = RunnableSequence.from([
  {
    context: retriever.pipe(formatDocs),
    question: new RunnablePassthrough(),
  },
  prompt,
  llm,
  new StringOutputParser(),
]);

```{=mdx}

:::提示

详见 [我们的RAG教程](docs/tutorials/rag) ，了解有关如上所示 `RunnableSequence` 的更多信息和示例。

:::

```

In [ ]:
await ragChain.invoke("...")

## API 参考

有关 AmazonKnowledgeBaseRetriever 所有功能和配置的详细文档，请访问 [API 参考](https://api.js.langchain.com/classes/langchain_aws.AmazonKnowledgeBaseRetriever.html)。